In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os

### Data preprocessing

In [2]:

image_size = (224, 224)
batch_size = 32

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [4]:
train_generator = datagen.flow_from_directory(
    "data/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training"
)

Found 160 images belonging to 2 classes.


In [5]:
val_generator = datagen.flow_from_directory(
    "data/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)

Found 40 images belonging to 2 classes.


### Hypermodel function

In [6]:
def build_cnn_model(hp):
    
    base_model = MobileNet(include_top=False, weights='imagenet', input_shape=image_size + (3,))
    base_model.trainable = False  # Transfer learning

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())

    # Hidden Dense Layer
    units = hp.Int("units", min_value=64, max_value=256, step=32)
    model.add(Dense(units=units,
                    activation=hp.Choice("activation", ['relu', 'tanh']),
                    kernel_initializer=hp.Choice("initializer", ['glorot_uniform', 'he_normal']),
                    kernel_regularizer=regularizers.l2(hp.Float("lambda", 1e-4, 1e-2, sampling='log'))))

    model.add(Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    # Optimizer
    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling='log')
    optimizer_choice = hp.Choice("optimizer", ['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

### Tuner setup

In [7]:
tuner = kt.RandomSearch(
    build_cnn_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='cnn_tuning',
    project_name='MobileNet_binary'
)

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 34s 2us/step


### Early stopping

In [8]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

### Tuner search

In [9]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop],
    verbose=2
)

Trial 10 Complete [00h 04m 16s]
val_loss: 2.2792086601257324

Best val_loss So Far: 0.15342199802398682
Total elapsed time: 00h 36m 24s


### Get best model and hyperparameters

In [10]:
best_model = tuner.get_best_models(1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

C:\Users\Tribhushan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
print("\nBest Hyperparameters:")
print(best_hyperparams.values)


Best Hyperparameters:
{'units': 64, 'activation': 'tanh', 'initializer': 'he_normal', 'lambda': 0.00038799848329565466, 'dropout': 0.30000000000000004, 'learning_rate': 0.000335981718179628, 'optimizer': 'rmsprop'}


In [12]:
# Save best model
os.makedirs("model", exist_ok=True)
best_model.save("model/MobileNet_best_tuned_model.h5")

### Evaluate Accuracy on Validation/Test Data

In [13]:
# Evaluate model on validation data
val_loss, val_accuracy = best_model.evaluate(val_generator, verbose=1)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 424ms/step - accuracy: 0.9833 - loss: 0.1455
Validation Accuracy: 97.50%
Validation Loss: 0.1495
